In [1]:
# jupyter notebook 환경에서 spark를 사용하기 위한 모듈
import findspark
findspark.init('C:/spark/spark-3.0.1-bin-hadoop2.7')

In [2]:
# pyspark import
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql import SparkSession

In [3]:
# pyspark 연결하기
# 꼬이면 kernel restart
conf= pyspark.SparkConf().setAppName('appName').setMaster('local')
sc= pyspark.SparkContext(conf= conf)
spark= SparkSession(sc)

In [4]:
# RDD 생성하기
data = sc.parallelize(
[('Amber',22), ('Alfred',23), ('Skye',4), ('Albert', 12),('Amber', 9)])

In [5]:
# 파일로부터 데이터를 받아오기
# data_from_file = sc.textFile('/Users/soohan/Documents/')

In [6]:
# 스키마
data_heterogeneous = sc.parallelize([
    ('Ferrari', 'fast'),
    {'Porsche': 10000},
    ['Spain', 'visited', 4504]
]).collect()

In [11]:
# 데이터셋에 대해 collect를 수행하면 일반적인 파이썬처럼 다룰 수 있다.
data_heterogeneous[1]['Porsche']
print(type(data_heterogeneous))

<class 'list'>
